# Sentiment Analysis
Sentiment analysis in the python file was not fast enough so now we are going to use the google cloud api and hope it works better. Currently need to get setup an account with google cloud with $300 credit, then turn on billing and add the nlp service. Follow these steps to use (NLP)[https://cloud.google.com/python/docs/reference/language/latest]. I then follow authentication on google api core which requires to install google cloud sdk for some reason. 

In [9]:
import numpy as np
import time
from google.cloud.language_v2 import types
from google.cloud import language_v2
import polars as pl
import os
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import seaborn as sns
import matplotlib.pyplot as plt
from google.auth import compute_engine

credentials = compute_engine.Credentials()

In [11]:
client = language_v2.LanguageServiceClient()

# Available types: PLAIN_TEXT, HTML
document_type_in_plain_text = language_v2.Document.Type.PLAIN_TEXT
language_code = "en"
encoding_type = language_v2.EncodingType.UTF8

def sentimentanalysis(text_content):
    document = {
        "content": text_content,
        "type_": document_type_in_plain_text,
        "language_code": language_code,
    }

    response = client.analyze_sentiment(
            request={"document": document, "encoding_type": encoding_type}
        )

    # Get overall sentiment of the input document
    print(f"Document sentiment score: {response.document_sentiment.score}")
    print(f"Document sentiment magnitude: {response.document_sentiment.magnitude}")
    return response.document_sentiment.score, response.document_sentiment.magnitude

C:\Users\Kyle\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
news_data = pl.read_parquet('news_data_preprocessed/nasdaq_external_data.parquet')
sentiment_scores = []
sentiment_magnitudes = []

for row in news_data.iter_rows():
    article = row[news_data.columns.index('Article')]
    if isinstance(article, str) and len(article.strip()) > 0:
        try:
            sentiment_score, sentiment_magnitude = sentimentanalysis(article)
        except Exception as e:
            print(f"Error processing article: {e}")
            sentiment_score, sentiment_magnitude = None, None
    
sentiment_scores.append(sentiment_score)
sentiment_magnitudes.append(sentiment_magnitude)

sentiment_df = pl.DataFrame({
    'Date': news_data['Date'],
    'Title': news_data['Title'],
    'Article': news_data['Article'],
    'Sentiment_Score': sentiment_scores,
    'Sentiment_Magnitude': sentiment_magnitudes
})

# Sort by date descending
sentiment_df = sentiment_df.sort('Date', descending=True)